In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import shutil
import os

output = os.path.join("output")
vidims = os.path.join(output, 'video')

def shutup(func):
    try:
        func()
    except:
        pass

def install_ffmpeg():
    !apt-get update
    !apt install -y ffmpeg
    !pip install ffmpeg-python

def clear_all():
    shutup(lambda: shutil.rmtree(output))
    shutup(lambda: os.remove('images.zip'))
    shutup(lambda: os.remove('animation.mp4'))
    
def create_folders():
    shutup(lambda: shutil.rmtree(output))
    shutup(lambda: os.mkdir(output))
    shutup(lambda: os.mkdir(vidims))
    
def renew_folders():
    clear_all()
    create_folders()
    
def create_zip():
    shutup(lambda: os.remove('images.zip'))
    shutil.make_archive('images', 'zip', output)

def create_video():
    shutup(lambda: os.remove('animation.mp4'))
    import ffmpeg
    ffmpeg.input(os.path.join(vidims, '*.png'), pattern_type='glob', framerate=2) \
        .output('animation.mp4', pix_fmt='yuv420p', vcodec='libx264').run()

#install_ffmpeg()    
renew_folders()
#create_zip()
#create_video()

In [3]:
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt

from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

In [4]:
try:
    # Ubuntu
    DrawFont = ImageFont.truetype('/opt/conda/fonts/UbuntuMono-R.ttf', 42)
except:
    # MacOS
    DrawFont = ImageFont.truetype('Arial.ttf', 42)

OSError: cannot open resource

In [ ]:
def show_image(img, size=(40, 6), title=None):
    plt.figure(figsize=size)
    plt.axis('off')
    
    if title is not None:
        plt.title(title)
    
    plt.imshow(img, cmap="gray", vmin=0, vmax=1)
    
    plt.show()
    return plt

def save_output(img, candidate, gen):
    name = f'gen{gen}'
    png = f'{name}.png'
    title = f'Gen {gen}'
    
    # Fix image
    img *= 255
    img = cv2.resize(img, (800, 800), interpolation=cv2.INTER_NEAREST)
    
    # Save original image
    cv2.imwrite(os.path.join(output, png), img)
    
    # Save candidate
    with open(os.path.join(output, f'{name}.txt'), 'wb') as f:
        np.savetxt(f, candidate.astype(int), fmt='%i')
    
    # Save image with text.
    img = np.array(img).astype('uint8')
    img = np.stack((img,) * 3, axis=-1)
    img = Image.fromarray(img, 'RGB')
    ImageDraw.Draw(img).text((10, 10), title, font=DrawFont, fill=(210, 0, 0))
    img.save(os.path.join(vidims, png))
    
def read_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img // 255
    return img

In [ ]:
#img_path = '/kaggle/input/genetic-homework/heart_20.png'
img_path = 'images/20/heart.png'

img = read_image(img_path)
show_image(img);

In [ ]:
# Moves
'''
8 1 2
7 0 3
6 5 4
'''
moves = [
    (0, 0), (-1, 0), (-1, 1),
    (0, 1), (1, 1), (1, 0),
    (1, -1), (0, -1), (-1, -1),
]

# Costs of rotations.
rotation_costs = [0, 1, 2, 4, 7, 4, 2, 1]
rot_cost_mean = sum(rotation_costs) // len(rotation_costs)
rot_count_prob = (np.array(rotation_costs) > 0).sum() / ( len(rotation_costs) + 1)

# Meta
non_empty_pixels = (img == 0).sum()

# Hyperparameters
candidate_size = int(non_empty_pixels * 1.5) # size of a candidate solution
pop_size = 4096 # candidate count per iteration
roll_mut_prob = 0.005 # probability of roll mutation
mut_prob = 0.07 # probability of mutation
mut_decay = 0.0007 # decay of mutation ratio per generation
max_gen = 2048 # maximum generation
cross_pick_strength = 2 # strengh of an fitness order to be picked to crossover
cross_point = 3 # split points of crossover
gen_keep = int(pop_size * 0.5) # number of best candidates to keep
gen_renew = pop_size - gen_keep
ft_round = 6

# Weights
weights = [
    1 * 1.5, # w_rotate_cost
    1 * 3.5, # w_overlap_loss
    1 * 2.5, # w_spread_loss
]

# Normalize weights.
weights /= np.array(weights).sum()

# Configurations
log_interval = 1
show_interval = 10

print(f'non_empty_pixels={non_empty_pixels}')
print(f'candidate_size={candidate_size}')
print(f'mutation_size={candidate_size * mut_prob}')
print(f'gen_keep={gen_keep}')
print(f'gen_renew={gen_renew}')

In [ ]:
# Create number random generator
rng = np.random.default_rng()

# Keep track of gen's best.
fit_of_gen_best = np.inf
gen_of_gen_best = 0
fit_of_last_best = np.inf

# Constants
side_size = img.shape[0]
bound_row = img.shape[0] - 1
bound_col = img.shape[1] - 1

In [ ]:
# Create initial population.
population = np.random.randint(1, 9, (pop_size, candidate_size))

# Iterate generations.
for gen in range(max_gen):
    start_time = time.time()
    fitnesses = np.full(pop_size, np.inf)
    
    # Keep best solution to show.
    best_fitness = np.inf
    best_candidate = None
    best_solution = None
        
    # Calculate fitness of each candidate
    for ci, candidate in enumerate(population):
        # Fitness of candidate
        ft_rotate_cost = 0
        ft_overlap_loss = 0
        ft_spread_loss = 0
        
        # Create solution of this candidate.
        solution = np.ones(img.shape)
        inverse_moves = 0
        
        # Start point for solution.
        prev_mov_dir = candidate[0]
        row = side_size - 1 
        col = 0
        solution[row][col] = 0
        
        for direction in candidate:
            # Calculate rotation cost.
            if direction != prev_mov_dir:
                rotate_costs = np.roll(rotation_costs, prev_mov_dir - 1)
                step_rotate_cost = rotate_costs[direction - 1]
                ft_rotate_cost += step_rotate_cost / rot_cost_mean
            prev_mov_dir = direction
            
            # Paint pixel if in bound.
            move = moves[direction]
            next_row = row + move[0]
            next_col = col + move[1]
            if (0 <= next_row <= bound_row) and (0 <= next_col <= bound_col):
                row = next_row
                col = next_col
                solution[row][col] = 0
        
        # Normalize rotation cost of candidate.
        ft_rotate_cost /= candidate_size

        # Calculate overlap rate fitness.
        overlap = (np.logical_or(solution, img) == False).sum()
        ft_overlap_loss = 1 - (overlap / non_empty_pixels)
        
        # Calculate spread loss fitness.
        ft_spread_loss = (non_empty_pixels - overlap) / non_empty_pixels
        
        # Calculate total fitness.
        losses = [ft_rotate_cost, ft_overlap_loss, ft_spread_loss]
        fitness = (weights * losses).sum()
        fitnesses[ci] = fitness

        # Keep if best candidate.
        if fitness < best_fitness:
            best_fitness = fitness
            best_solution = solution
            best_candidate = candidate
    
    # Order the candidates by their fitness.
    fitnesses = np.round(fitnesses, ft_round)
    orders = fitnesses.argsort()
    fitnesses = fitnesses[orders]
    population = population[orders] 
    
    # Scale fitnesses.
    # Use order selection with pick strength.
    ft_scaled = np.arange(1, (pop_size * cross_pick_strength + 1), cross_pick_strength) 
    
    # Calculate weight distribution of candidates.
    ft_probs = 1 / ft_scaled
    ft_probs = ft_probs / ft_probs.sum()
    
    # Create 2n pairs by weighted random (fitness)
    pairs_len = pop_size + 1
    pairs = population[np.random.choice(population.shape[0], pairs_len, p=ft_probs)]
    
    # Crossover and Mutate to create new candidates.
    new_population = []
    for i in range(0, pairs_len - 1):
        # Crossover 
        a = pairs[i]
        b = pairs[i+1]
        # Generate random split points.
        splits = rng.choice(candidate_size, size=cross_point, replace=False)
        splits.sort()
        # Split parents.
        a = np.split(a, splits, axis=0)
        b = np.split(b, splits, axis=0)
        # Pick parts randomly from parents.
        pair = [a, b]
        picks = np.random.randint(0, 2, cross_point + 1)
        new_a = [x for i, p in enumerate(picks) for x in pair[p][i]]
        new_b = [x for i, p in enumerate(1 - picks) for x in pair[p][i]]
        
        # Add candidate to new population.
        new_population.append(new_a)
        new_population.append(new_b)
        
    # Keep some of current generation and fill rest with new.
    population[gen_keep:] = new_population[:gen_renew]
    
    # Mutate
    gen_mut_prob = mut_prob * ((1 - mut_decay) ** gen)
    mutation_points = np.random.rand(pop_size, candidate_size) < gen_mut_prob
    mutations = np.random.randint(1, 9, (pop_size, candidate_size))
    population[mutation_points] = mutations[mutation_points]
       
    # Stats for current gen.
    last_gen_best = fit_of_gen_best
    curr_gen_best = fitnesses[0]
    
    # Log generation stats.
    if gen % log_interval == 0:
        print(
            f'Gen#{gen} ft={curr_gen_best} {round(time.time() - start_time)}s '
            f'(last same={curr_gen_best == fit_of_last_best} ft={fit_of_last_best}) '
            f'(best same={curr_gen_best == fit_of_gen_best} gen={gen_of_gen_best} ft={fit_of_gen_best})'
        )
    
    # Show the best candidate.
    if gen % show_interval == 0:
        show_image(best_solution)
        save_output(best_solution, best_candidate, gen)
        
    # Update stats.
    fit_of_last_best = curr_gen_best
    if curr_gen_best < fit_of_gen_best:
        fit_of_gen_best = curr_gen_best
        gen_of_gen_best = gen

# Show final results.
print('\nDone.')
print(f'lastft={fit_of_last_best} bestft={fit_of_gen_best} bestgen={gen_of_gen_best}')
show_image(best_solution)
save_output(best_solution, best_candidate, 'best')